# ДЗ1: Извлечение ключевых слов

## 1. Создание мини-корпуса + 2. Самостоятельная разметка ключевых слов

Корпус создан на основе трудов международной конференции «Корпусная лингвистика-2021» ([ссылка на сборник](https://elibrary.ru/item.asp?id=47945360)). Ключевые слова и тексты были скопированы из pdf-файла вручную. Из каждого текста были взяты только первая и последняя части (условно Введение и Заключение) для простоты анализа.

In [165]:
corp_ling_dict = {
    '1': {
        'article_keywords': ['нкря', 'состав текстов', 'репрезентативность', 'сбалансированность'],
        'my_keywords': ['нкря', 'корпус', 'репрезентативность', 'сбалансированность', 'объем корпуса'],
        'text': ''
    },
    '2': {
        'article_keywords': ['синтаксис зависимостей', 'анафорическая разметка', 'лексико-функциональная разметка', 
                     'микросинтаксическая разметка', 'темпоральная разметка', 'эллипсис', 'поиск'],
        'my_keywords': ['СинТагРус', 'разметка', 'доступность'],
        'text': ''
    },
    '3': {
        'article_keywords': ['поверхностный синтаксический анализ', 'автоматическое извлечение конструкций', 'русский язык'],
        'my_keywords': ['синтаксический анализ', 'конструкции', 'метод', 'русский язык'],
        'text': ''
    },
    '4': {
        'article_keywords': ['концепт', 'дистрибутивный тезаурус', 
                     'сопоставительное исследование', 'языковая картина мира'],
        'my_keywords': ['концепт', 'языковая картина мира', 'государство', 'лексема'],
        'text': ''
    },
    '5': {
        'article_keywords': ['коллокации', 'база данных', 'словари', 'статистические методы', 'русский язык'],
        'my_keywords': ['лексическая сочетаемость', 'коллокации', 'база данных'],
        'text': ''
    },
    '6': {
        'article_keywords': ['гендерный стереотип', 'гендерная идентичность', 'корпусная лингвистика', '«Твиттер»'],
        'my_keywords': ['блог', 'образ', 'стереотип', 'твиты', 'женщины-политики'],
        'text': ''
    },
    '7': {
        'article_keywords': ['антропоморфные метафоры', 'китайский язык', 'русский язык', 'корпусный анализ'],
        'my_keywords': ['китайский язык', 'русский язык', 'политический дискурс', 'метафора'],
        'text': ''
    },
}

In [166]:
num_tokens = 0
for i in corp_ling_dict.keys():
    with open(f'corpus/{i}.txt', encoding='utf-8') as f:
        text = f.read().replace('-\n', '').replace('\xa0', ' ').replace('\n', ' ')
    corp_ling_dict[i]['text'] = text
    corp_ling_dict[i]['keywords'] = set(corp_ling_dict[i]['article_keywords']) | set(corp_ling_dict[i]['my_keywords'])
    num_tokens += len(text.split())

In [167]:
print(f'В корпусе {num_tokens} токенов')

В корпусе 3114 токенов


## 3. Методы извлечения ключевых слов

### 3.1. RAKE

In [28]:
# установка
!pip3 install python-rake

In [168]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = stopwords.words('russian') + ['это', 'то', 'ты', 'мы', 'вы', 'я', 'он', 'она', 'оно', 'они']
rake = RAKE.Rake(stops)

[nltk_data] Downloading package stopwords to /Users/tasia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [169]:
for i in corp_ling_dict.keys():
    rake_kw_list = rake.run(corp_ling_dict[i]['text'], maxWords=3, minFrequency=2)
    corp_ling_dict[i]['rake_kw'] = [w for w, i in rake_kw_list]

In [170]:
corp_ling_dict['3']['keywords'], corp_ling_dict['3']['rake_kw']

({'автоматическое извлечение конструкций',
  'конструкции',
  'метод',
  'поверхностный синтаксический анализ',
  'русский язык',
  'синтаксический анализ'},
 ['методы машинного обучения',
  'др',
  'см',
  'например',
  '[большакова',
  'заметим'])

In [171]:
# чтобы было получше -- почистим
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize

known_words = dict()
m = MorphAnalyzer()

In [172]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if t.isalpha():
            if t in known_words:
                lemm = known_words[t]
            else:
                lemm = m.parse(t)[0].normal_form
                known_words[t] = lemm
            lemmas.append(lemm)
    return ' '.join(lemmas)

In [226]:
for i in corp_ling_dict.keys():
    corp_ling_dict[i]['norm_text'] = normalize_text(corp_ling_dict[i]['text'])
    rake_kw_list = rake.run(corp_ling_dict[i]['norm_text'], maxWords=3, minFrequency=2) 
    corp_ling_dict[i]['rake_kw'] = set([w for w, i in rake_kw_list[:5]])

In [227]:
corp_ling_dict['3']['keywords'], corp_ling_dict['3']['rake_kw']
# ээээ у не то чтобы стало лучше...

({'автоматическое извлечение конструкций',
  'конструкции',
  'метод',
  'поверхностный синтаксический анализ',
  'русский язык',
  'синтаксический анализ'},
 {'синтаксический связь'})

### 3.2. TextRank

In [82]:
!pip3 install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 693.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54411 sha256=9fe8671093ad85d15e314d30e07e31fcccc89da4b9639e207e4aead4b5b6fda2
  Stored in directory: /Users/tasia/Library/Caches/pip/wheels/ed/2c/5f/a0ccc5955d44d2cea78729f4425e73f818d2629517f7af0f8b
Successfully built summa


In [83]:
from summa import keywords

In [175]:
for i in corp_ling_dict.keys():
    tr_kw_string = keywords.keywords(corp_ling_dict[i]['norm_text'], language='russian', 
                              additional_stopwords=stops, scores=False)
    corp_ling_dict[i]['tr_kw'] = set(tr_kw_string.split('\n')[:5])

In [176]:
corp_ling_dict['3']['keywords'], corp_ling_dict['3']['tr_kw']

({'автоматическое извлечение конструкций',
  'конструкции',
  'метод',
  'поверхностный синтаксический анализ',
  'русский язык',
  'синтаксический анализ'},
 {'конструкция',
  'метод весь',
  'поиск синтаксически',
  'проведение синтаксический анализ текст являться',
  'работа'})

### 3.3. TF-IDF

In [177]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

In [178]:
all_texts = [corp_ling_dict[i]['norm_text'] for i in corp_ling_dict.keys()]

In [179]:
tfidf_matrix = tfidf_vectorizer.fit_transform(all_texts)
feature_names = np.array(tfidf_vectorizer.get_feature_names())

In [180]:
tfidf_matrix.shape

(7, 6030)

In [181]:
from collections import defaultdict

In [182]:
# попытка в сортировку спарс без перевода в денс, ибо а в чем тогда смысл
rows, cols = tfidf_matrix.nonzero()

idx = defaultdict(list)
for r, c, v in zip(rows, cols, tfidf_matrix.data):
    idx[r].append((c, v))

for r in idx.keys():
    sorted_row = sorted(idx[r], key=lambda v: v[1], reverse=True)
    idx[r] = [i for i, v in sorted_row[:5]]
    corp_ling_dict[str(r + 1)]['tfidf_kw'] = set(feature_names[idx[r]].flatten())

In [183]:
corp_ling_dict['3']['keywords'], corp_ling_dict['3']['tfidf_kw']

({'автоматическое извлечение конструкций',
  'конструкции',
  'метод',
  'поверхностный синтаксический анализ',
  'русский язык',
  'синтаксический анализ'},
 {'конструкция',
  'метод',
  'синтаксический',
  'синтаксический анализ',
  'точность'})

## 4. Морфологические шаблоны

В беседе в тг сказали, что можно уже из готовых keywords оставлять те, что удовлетворяют шаблонам. Но в моем случае keywords состоят уже из лемматизированных слов без дополнительной разметки. Поэтому я буду использовать только частеречные шаблоны (хотя в целом даже на основе такой информации можно делать выводы о синтаксических связях, см. мою с ЭС статью в сборнике по ссылке выше).

In [228]:
for t in ['rake', 'tr', 'tfidf']:
    for i in corp_ling_dict.keys():
        kws = corp_ling_dict[i][f'{t}_kw']
        kw_templates = []
        
        for kw in kws:
            kw_ = kw.split()
            if len(kw_) == 2:
                kw1_pos = m.parse(kw_[0])[0].tag.POS
                kw2_pos = m.parse(kw_[1])[0].tag.POS

                # ADJ + NOUN
                if (kw1_pos == 'ADJF') and (kw2_pos == 'NOUN'):
                    kw_templates.append(kw)

                # NOUN + NOUN (тип генетив)
                elif (kw1_pos == 'NOUN') and (kw2_pos == 'NOUN'):
                    kw_templates.append(kw)
        
        print(kw_templates)
        corp_ling_dict[i][f'{t}_kw_templates'] = set(kw_templates)
# ну такое...

[]
[]
['синтаксический связь']
[]
[]
['результат исследование']
['политический дискурс', 'русский язык']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['синтаксический анализ']
['картина мир', 'языковой картина']
[]
[]
['политический дискурс']


## 5. Точность, полнота, F-мера

Я не нашла в питоне готовых метрик в данном значении (только в контексте бинарной классификации), поэтому, руководствуясь [соответствующим разделом Википедии](https://en.wikipedia.org/wiki/Precision_and_recall#Definition_(information_retrieval_context)), ниже задала функции.

In [213]:
def precision(set_new, set_gold):
    try:
        pr = len(set_new & set_gold) / len(set_new)
    except ZeroDivisionError:
        pr = 0
    return pr

def recall(set_new, set_gold):
    try: 
        rec = len(set_new & set_gold) / len(set_gold)
    except ZeroDivisionError:
        rec = 0
    return rec
    
def f1_score(set_new, set_gold):
    try:
        pr = precision(set_new, set_gold)
        rec = recall(set_new, set_gold)
        f1 = 2 * pr * rec / (pr + rec)
    except ZeroDivisionError:
        f1 = 0
    return f1

In [210]:
import pandas as pd

In [222]:
# для более честного сравнения слова из эталона тоже надо леммантизировать
for i in corp_ling_dict.keys():
    kw_norm = []
    for kw in corp_ling_dict[i]['keywords']:
        kw_norm.append(normalize_text(kw))
    corp_ling_dict[i]['keywords_norm'] = set(kw_norm)

In [219]:
!pip3 install tabulate

In [229]:
for i in corp_ling_dict.keys():
    print(f'Text #{i}')
    df_results = pd.DataFrame({'Metric': ['precision', 'recall', 'F1-score']})
    df_results = df_results.set_index('Metric')
    
    for t1 in ['rake', 'tr', 'tfidf']:
        for t2 in ['', '_templates']:
            #print(corp_ling_dict[i][f'{t1}_kw{t2}'], corp_ling_dict[i]['keywords_norm'])
            pr = precision(corp_ling_dict[i][f'{t1}_kw{t2}'], corp_ling_dict[i]['keywords_norm'])
            rec = recall(corp_ling_dict[i][f'{t1}_kw{t2}'], corp_ling_dict[i]['keywords_norm'])
            f1 = f1_score(corp_ling_dict[i][f'{t1}_kw{t2}'], corp_ling_dict[i]['keywords_norm'])
            df_results[f'{t1}_kw{t2}'] = [pr, rec, f1]
    print(df_results.to_markdown())
    print('\n\n')

Text #1
| Metric    |   rake_kw |   rake_kw_templates |    tr_kw |   tr_kw_templates |   tfidf_kw |   tfidf_kw_templates |
|:----------|----------:|--------------------:|---------:|------------------:|-----------:|---------------------:|
| precision |         0 |                   0 | 0.2      |                 0 |   0.2      |                    0 |
| recall    |         0 |                   0 | 0.166667 |                 0 |   0.166667 |                    0 |
| F1-score  |         0 |                   0 | 0.181818 |                 0 |   0.181818 |                    0 |



Text #2
| Metric    |   rake_kw |   rake_kw_templates |    tr_kw |   tr_kw_templates |   tfidf_kw |   tfidf_kw_templates |
|:----------|----------:|--------------------:|---------:|------------------:|-----------:|---------------------:|
| precision |         0 |                   0 | 0.2      |                 0 |   0.4      |                    0 |
| recall    |         0 |                   0 | 0.111111 |   

**Комментарии к результатам:**
* Лучше всего с задачей справляется TF-IDF без шаблонов: в трех случаях дает такие же результаты как TextRank/RAKE, а в четырех показывает более высокие метрики
* RAKE почти везде ничего нужного не выделил, только в одном из семи текстов (любопытно, что это также единственный текст, на котором TextRank не заработал, но прокомментировать эту дополнительную дистрибуцию пока не могу)
* добавление морфологических шаблонов в целом только ухудшает ситуацию (не остается кандидатов на keywords), при этом в тех случаях, где шаблоны что-то оставили, поднимается precision, но подают recall и F1

## 6. Обсуждение ошибок

Рассмотрим ошибки на примере текстов 1, 5, 7.

In [233]:
print('Эталон:', corp_ling_dict['1']['keywords'], end='\n\n')
print('Эталон (нормализованный):', corp_ling_dict['1']['keywords_norm'], end='\n\n')
for t1 in ['rake', 'tr', 'tfidf']:
    for t2 in ['', '_templates']:
        print(f'{t1} ({t2}):', corp_ling_dict['1'][f'{t1}_kw{t2}'], end='\n\n')

Эталон: {'нкря', 'корпус', 'репрезентативность', 'объем корпуса', 'сбалансированность', 'состав текстов'}

Эталон (нормализованный): {'корпус', 'репрезентативность', 'объесть корпус', 'нкрить', 'сбалансированность', 'состав текст'}

rake (): {'половина xix', 'время'}

rake (_templates): set()

tr (): {'который', 'репрезентативность', 'период', 'национальный корпус русский язык', 'текст'}

tr (_templates): set()

tfidf (): {'корпус', 'текст', 'словоупотребление', 'миллион', 'объём'}

tfidf (_templates): set()



**Комментарий (часть 1):**
* проблемы с аббревиатурами и их полным написанием (глагол *нкрить*, до слез)
* проблемы со стоп-словами (попали, например, союзы)
* проблемы с частыми словами для корпуса тестов (хотя я скорее ожидала, что наоборот, такие слова уйдут из-за особенностей коллекции, но, как мы видим,метрики всё равно выдвигают слова *корпус* и *текст*)
* везде сравнивалась с лемматизированным эталоном, но, конечно, пользователю такое выдавать нельзя -- надо обратно изменять слова, согласовывать между собой, а это отдельная непростая задача

In [236]:
print('Эталон:', corp_ling_dict['3']['keywords'], end='\n\n')
print('Эталон (нормализованный):', corp_ling_dict['3']['keywords_norm'], end='\n\n')
for t1 in ['rake', 'tr', 'tfidf']:
    for t2 in ['', '_templates']:
        print(f'{t1} ({t2}):', corp_ling_dict['3'][f'{t1}_kw{t2}'], end='\n\n')

Эталон: {'поверхностный синтаксический анализ', 'синтаксический анализ', 'метод', 'автоматическое извлечение конструкций', 'конструкции', 'русский язык'}

Эталон (нормализованный): {'поверхностный синтаксический анализ', 'конструкция', 'русский язык', 'автоматический извлечение конструкция', 'синтаксический анализ', 'метод'}

rake (): {'синтаксический связь'}

rake (_templates): {'синтаксический связь'}

tr (): {'проведение синтаксический анализ текст являться', 'метод весь', 'конструкция', 'работа', 'поиск синтаксически'}

tr (_templates): set()

tfidf (): {'конструкция', 'синтаксический анализ', 'синтаксический', 'метод', 'точность'}

tfidf (_templates): {'синтаксический анализ'}



**Комментарий (часть 2):**
+ проблема с трехсловными ключевыми словами, их не достает никтоб в отличие от двухсловных (ср. *поверхностный синтаксический анализ* и *синтаксический анализ*)
* лёгкие глаголы (*являться*), кажется, тоже можно занести в стоп-слова

In [235]:
print('Эталон:', corp_ling_dict['7']['keywords'], end='\n\n')
print('Эталон (нормализованный):', corp_ling_dict['7']['keywords_norm'], end='\n\n')
for t1 in ['rake', 'tr', 'tfidf']:
    for t2 in ['', '_templates']:
        print(f'{t1} ({t2}):', corp_ling_dict['7'][f'{t1}_kw{t2}'], end='\n\n')

Эталон: {'метафора', 'китайский язык', 'политический дискурс', 'русский язык', 'антропоморфные метафоры', 'корпусный анализ'}

Эталон (нормализованный): {'метафора', 'китайский язык', 'политический дискурс', 'русский язык', 'корпусный анализ', 'антропоморфный метафора'}

rake (): {'тема', 'политический дискурс', 'путь', 'русский язык', 'пояс'}

rake (_templates): {'политический дискурс', 'русский язык'}

tr (): {'анализ', 'политический', 'китайский', 'антропоморфный', 'проект'}

tr (_templates): set()

tfidf (): {'метафора', 'политический дискурс', 'дискурс', 'политический', 'антропоморфный'}

tfidf (_templates): {'политический дискурс'}



**Комментарий (часть 3):**
* иногда находится подчасть ключевого слова (ср. *антропоморфная метафора* и *антропоморфный*)

**Предложения по улучшению:**
* тщательнее составлять список стоп-слов (добавлять туда союзы, легкие глаголы)
* ограничивать длину ключевого слова двумя токенами
* мб зашитывать за "полбалла", если найдена подстрока ключевого слова
* предварительный этап NER и установления синонимии для уменьшения размера словаря и повышения частотностей ключевых слов